In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import mysql.connector
import folium

In [3]:
# AWS INSTANCE에서 데이터 가져오기

yespa_db = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'eda_team',
    database = 'eda_project',
    password = '123456'
)

cur = yespa_db.cursor(buffered=True, dictionary=True)
cur.execute('SELECT * FROM GIHU_metro_coor')

result1 = cur.fetchall()
for data in result1:
    print(data)

cur.execute('SELECT * FROM GIHU_metro_colors')

result2 = cur.fetchall()
for data in result2:
    print(data)
    
yespa_db.close()

{'역명': '창동', '호선': '1호선', '위도': 37.653181, '경도': 127.047663}
{'역명': '월계', '호선': '1호선', '위도': 37.633114, '경도': 127.058835}
{'역명': '외대앞', '호선': '1호선', '위도': 37.596157, '경도': 127.063572}
{'역명': '대방', '호선': '1호선', '위도': 37.513392, '경도': 126.926407}
{'역명': '신길', '호선': '1호선', '위도': 37.517018, '경도': 126.917743}
{'역명': '구일', '호선': '1호선', '위도': 37.496273, '경도': 126.869599}
{'역명': '온수', '호선': '1호선', '위도': 37.492267, '경도': 126.823399}
{'역명': '소요산', '호선': '1호선', '위도': 37.5653, '경도': 127.0339}
{'역명': '도봉산', '호선': '1호선', '위도': 37.689603, '경도': 127.046347}
{'역명': '방학', '호선': '1호선', '위도': 37.66796, '경도': 127.04456}
{'역명': '녹천', '호선': '1호선', '위도': 37.644599, '경도': 127.051406}
{'역명': '석계', '호선': '1호선', '위도': 37.615206, '경도': 127.065594}
{'역명': '신이문', '호선': '1호선', '위도': 37.60274474, '경도': 127.067556}
{'역명': '회기', '호선': '1호선', '위도': 37.589802, '경도': 127.057936}
{'역명': '남영', '호선': '1호선', '위도': 37.540597, '경도': 126.971171}
{'역명': '용산', '호선': '1호선', '위도': 37.52989, '경도': 126.964775}
{'역명': '노량진', '호선': '1호선'

In [4]:
df1 = pd.DataFrame(result1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      406 non-null    object 
 1   호선      406 non-null    object 
 2   위도      406 non-null    float64
 3   경도      406 non-null    float64
dtypes: float64(2), object(2)
memory usage: 12.8+ KB


In [5]:
df2 = pd.DataFrame(result2)
df2

,호선,색상
0,1호선,darkblue
1,2호선,lightgreen
2,3호선,orange
3,4호선,lightblue
4,5호선,darkpurple
5,6호선,darkred
6,7호선,darkgreen
7,8호선,pink
8,9호선,lightgray
9,우이신설선,lightgreen


In [6]:
numlist=[]

for 호선 in sorted(df1['호선'].unique()):
    count = (df1['호선'] == 호선).sum()
    print(f"{호선}은 {count}개")
    numlist.append(count)

1호선은 35개
2호선은 50개
3호선은 34개
4호선은 26개
5호선은 52개
6호선은 39개
7호선은 42개
8호선은 18개
9호선은 38개
경의중앙은 22개
경춘은 7개
공항철도은 6개
수인분당은 13개
신림은 11개
우이신설선은 13개


In [7]:
import folium

m = folium.Map(location=[37.58, 127.0], zoom_start=10)

for i in range(len(numlist)):
    for j in range(sum(numlist[:i]), sum(numlist[:i+1])):
        icon_number = str(i+1) if i < 9 else ''
        folium.Marker([df1['위도'][j], df1['경도'][j]], 
                      icon=folium.Icon(color=df2['색상'][i], icon=icon_number, prefix='fa'),
                      popup=df1['역명'][j]).add_to(m)

m

/tmp/ipykernel_5290/1600419874.py:9: UserWarning: color argument of Icon should be one of: {'beige', 'darkgreen', 'gray', 'red', 'lightred', 'pink', 'blue', 'darkblue', 'lightgreen', 'cadetblue', 'orange', 'green', 'black', 'darkpurple', 'darkred', 'lightgray', 'white', 'lightblue', 'purple'}.
  icon=folium.Icon(color=df2['색상'][i], icon=icon_number, prefix='fa'),


In [8]:
df_standard = df1[df1['역명'] == '가산디지털단지']
df_standard.reset_index(drop=True, inplace=True)
df_standard

,역명,호선,위도,경도
0,가산디지털단지,1호선,37.481568,126.882628
1,가산디지털단지,7호선,37.480376,126.882704


In [9]:
def folium_gasan_station(m):
    folium.Marker([df_standard['위도'][1],df_standard['경도'][1]], 
                icon =folium.Icon(icon= 'star', prefix='fa',color=df2['색상'][6])
                ).add_to(m)
    folium.Marker([df_standard['위도'][0],df_standard['경도'][0]], 
                icon =folium.Icon(icon= 'star', prefix='fa',color=df2['색상'][0])
                ).add_to(m)
    

In [10]:
folium_gasan_station(m)

# Calculate the distance in degrees for 1 km
# 1 degree of latitude is approximately 111.32 km and 1 degree of longitude is approximately 111.32 km * cos(latitude)
# We'll use average latitude for simplicity
one_km_in_degrees_lat = 1 / 111.32
one_km_in_degrees_lng = 1 / (111.32 * 111.32)

# Add a circle with a fixed radius of 1km
folium.Circle(
    location=[df_standard['위도'][1],df_standard['경도'][1]],
    radius=10000,  # 1 km in meters
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.2,
    popup='1 km radius',
).add_to(m)
# folium.CircleMarker(
#     location=[df_standard['위도'][1],df_standard['경도'][1]],
#     radius=100,
#     fill=True,
#     color="#dddddd",
#     fill_color="red",
#     popup="CircleMarker",
#     tooltip="CircleMarker tooltip"
# ).add_to(m)
m


In [20]:
import folium

m = folium.Map(location=[37.58, 127.0], zoom_start=10)

for i in range(len(numlist)):
    for j in range(sum(numlist[:i]), sum(numlist[:i+1])):
        icon_number = str(i+1) if i < 9 else 'info-sign'
        folium.Marker([df1['위도'][j], df1['경도'][j]], 
                      icon=folium.Icon(color=df2['색상'][i], icon=icon_number, prefix='fa'),
                      popup=df1['역명'][j]).add_to(m)
folium.Circle(
    location=[df_standard['위도'][1],df_standard['경도'][1]],
    radius=15000,  # 15 km in meters
    color='lightred',
    fill=True,
    fill_color='red',
    fill_opacity=0.2,
    popup='15km radius',
).add_to(m)
folium.Circle(
    location=[df_standard['위도'][1],df_standard['경도'][1]],
    radius=20000,
    color='lightyellow',
    fill_opacity=0.5,
    popup='20km radius',
).add_to(m)
folium.Circle(
    location=[df_standard['위도'][1],df_standard['경도'][1]],
    radius=25000, 
    color='lightgreen',
    fill_opacity=0.5,
    popup='25km radius',
).add_to(m)

folium.Circle(
    location=[df_standard['위도'][1],df_standard['경도'][1]],
    radius=30000,  
    color='lightblue',
    fill_opacity=0.5,
    popup='30km radius',
).add_to(m)

m

/tmp/ipykernel_5290/2579121590.py:9: UserWarning: color argument of Icon should be one of: {'beige', 'darkgreen', 'gray', 'red', 'lightred', 'pink', 'blue', 'darkblue', 'lightgreen', 'cadetblue', 'orange', 'green', 'black', 'darkpurple', 'darkred', 'lightgray', 'white', 'lightblue', 'purple'}.
  icon=folium.Icon(color=df2['색상'][i], icon=icon_number, prefix='fa'),
